<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://drive.google.com/uc?export=view&id=1LqkEgbpZj8A99Y9T59eBp9fEIZbpg6P2" alt="Snowflake Snowpark Classroom" style="width: 800px">
</div>

# Classroom 1.1 - Understanding Snowpark Session

In this notebook, you will learn to understand how to establish connectivity with snowflake using Snowpark for Python on Jupyter Notebook with VSCode

## Learning Objectives

By the end of this classroom, you should be able to:
- Get understanding on Snowpark Session Method
- Establishing the Connectivity with Snowflake using Snowpark Session Builder
- Setting up the session level configuration for your snowpark coding journey 

In [ ]:
# Getting Started With Snowpark
from snowflake.snowpark.session import Session

## Section 1 - Snowpark Session Builder


Snowpark Library uses Python Connector to connect to Snowflake. This is done through the snowpark session object.

To establish the snowflake connection, you require a Session Object. You must provide connection parameters to connect with a Snowflake database.

Builder attribute of session object allows to set configurations and connectivity with snowflake

Important thing to note here is that session object is not a thread-safe object

In [ ]:
from assets.config import session_config

session_config_local = {
    "account": "XXXXXXXXX.ap-southeast-1",
    "user": "XXXXXXXXX",
    "role": "XXXXXXXXX",
    "warehouse": "XXXXXXXXX",
    "database": "XXXXXXXXX",
    "schema": "XXXXXXXXX",
    "password": "XXXXXXXXX",
}

session = Session.builder.configs(session_config).create()

## Section 2 - Adding Libaries to Snowpark Session

You can import libraries using the Session.add_import()

The local file can be a compressed file (e.g., zip), a Python file (.py), a directory, or any other file resource.

In [ ]:
from assets.misc import sum_val
session.add_import('assets/misc.py','misc')

## Section 3.1 - Adding Packages to Snowpark Session

We can include third party packages as dependencies of a user defined function using Session.add_packages()

We can use this method to add packages for UDFs as installing packages using conda.


In [ ]:
import numpy as np
from snowflake.snowpark.functions import udf

session.add_packages(np)

@udf
def get_numpy_data() -> list:
    return [np.__name__]

session.sql(f"select {get_numpy_data.name}()").to_df('col').show()


## Section 3.2 - Adding Packages to Snowpark Session via requirements file

With the help of Session.add_requirements(), we can add all packages that contains a list of packages as dependencies of a user-defined function (UDF)

In [ ]:
# Lets try how we can import the list of packages from a requirements list
session.add_requirements(file_path='assets/requirements.txt')

## Section 4 - Getting Session Details using Snowpark Session methods

Snowflake Snowpark API provides methods to extract the current active session context details. This includes:

| Snowpark Method | Description |
|--|--|
| Session.get_current_account() | Returns the name of the current account for the Python connector session attached to this session |
| Session.get_current_database() | Returns the name of the current database for the Python connector session attached to this session |
| Session.get_current_role() | Returns the name of the primary role in use for the current session |
| Session.get_current_schema() | Returns the name of the current schema for the Python connector session attached to this session |
| Session.get_current_warehouse() | Returns the name of the warehouse in use for the current session |
| Session.get_fully_qualified_current_schema() | Returns the fully qualified name of the current schema for the session |
| Session.get_imports() | Returns a list of imports added for user defined functions (UDFs). This list includes any Python or zip files that were added automatically by the library |
| Session.get_packages() | Returns a dict of packages added for user-defined functions (UDFs). The key of this dict is the package name and the value of this dict is the corresponding requirement specifier |
| Session.get_session_stage() | Returns the name of the temporary stage created by the Snowpark library for uploading and storing temporary artifacts for this session. |



In [ ]:
print("Account Name                     :"+ session.get_current_account())
print("Database Name                    :"+ session.get_current_database())
print("Role Name                        :"+ session.get_current_role())
print("Schema Name                      :"+ session.get_current_schema())
print("Warehouse Name                   :"+ session.get_current_warehouse())
print("Fully Qualified Schema Name      :"+ session.get_fully_qualified_current_schema())
print("Imports Name                     :"+ str(session.get_imports()))
print("Packages Name                    :"+ str(session.get_packages()))
print("Stage Name                       :"+ session.get_session_stage())

## Section 5 - Setting Up Snowflake Session Context using Snowpark API

Snowflake Snowpark API provides methods to specifies the session context.

|Snowpark Method| Description |
|--|--|
|Session.use_database()|Specifies the active/current database for the session|
|Session.use_role()|Specifies the active/current primary role for the session|
|Session.use_schema()|Specifies the active/current schema for the session|
|Session.use_secondary_roles()|Specifies the active/current secondary roles for the session. The currently-active secondary roles set the context that determines whether the current user has the necessary privileges to perform SQL actions|
|Session.use_warehouse()|Specifies the active/current warehouse for the session|


In [ ]:
session.use_database('SNOWFLAKE')
session.use_role('sysadmin')
session.use_schema('information_schema')
session.use_secondary_roles('all')
session.use_warehouse('compute_wh')


print("Database Name                    :"+ session.get_current_database())
print("Role Name                        :"+ session.get_current_role())
print("Schema Name                      :"+ session.get_current_schema())
print("Warehouse Name                   :"+ session.get_current_warehouse())

## Section 6 - Cleaning Up Session's Context and Configurations

While you have Session methods to add imports or third party packages, Snowflake Snowpark API also provides the methods to remove or clear the packages or imports. Also You can terminate the entire session using Session.close()

|Snowpark Method | Description | 
|--|--|
|Session.remove_import()|Removes a file in stage or local file from the imports of a user-defined function|
|Session.remove_package()|Removes a third-party package from the dependency list of a user-defined function|
|Session.clear()|Removes a file in stage or local file from the imports of a user-defined function|
|Session.clear_imports()|Clears all files in a stage or local files from the imports of a user-defined function|
|Session.clear_packages()|Clears all files in a stage or local files from the packages of a user-defined function|
|Session.close()|Terminate the snowflake Session|

In [18]:
session.add_import('assets/config.py','config')
session.add_import('assets/misc.py','misc')
session.add_requirements('assets/requirements.txt')


print('-----------------------------')
print(session.get_imports())
print(session.get_packages())

print('-----------------------------\n Removing the openai from packages and config from imports\n')
session.remove_import('assets/config.py')
session.remove_package('openai')
print(session.get_imports())
print(session.get_packages())

print('-----------------------------\n Clearing all packages and imports from the session\n')
session.clear_imports()
session.clear_packages()
print(session.get_imports())
print(session.get_packages())

print('-----------------------------\n terminating the snowflake session\n')
session.close()
print('Session has been terminated now')


The version of package openai in the local environment is 0.27.2, which does not fit the criteria for the requirement openai. Your UDF might not work when the package version is different between the server and your local environment


-----------------------------
['c:\\Users\\divya\\Downloads\\Data Engineering With Snowflake\\01 - Snowpark API Commands\\assets\\config.py', 'c:\\Users\\divya\\Downloads\\Data Engineering With Snowflake\\01 - Snowpark API Commands\\assets\\misc.py']
{'pandas': 'pandas', 'openai': 'openai'}
-----------------------------
 Removing the openai from packages and config from imports

['c:\\Users\\divya\\Downloads\\Data Engineering With Snowflake\\01 - Snowpark API Commands\\assets\\misc.py']
{'pandas': 'pandas'}
-----------------------------
 Clearing all packages and imports from the session

[]
{}
-----------------------------
 terminating the snowflake session

Session has been terminated now
